## Лабораторная работа 4: topic modeling

В данной лабораторной работе мы попытаемся обучить LDA-модель topic-моделингу на двух принципиально различных корпусах. 

В первой части вы познакомитесь с новыми возможностями библиотеки gensim, а также с возможностями парсинга в языке Python. Во второй части вам предстоит самостоятельно обучить LDA-модель и оценить качество её работы.

### Часть 1: topic modeling уровня /b/

Краеугольным камнем в машинном обучений в целом, и в NLP в частности, является выбор датасетов. Доселе мы использовали только стандартные, многократно обкатанные датасеты, но сегодня попробуем собрать свой. Практика работы с сырыми, необработанными данными весьма полезно! Заодно изучим возможности парсеров в Питоне.

Давайте напишем парсер, собирающий информацию о сообщения с русскомязычного анононимного форума (имиджборды) "Двач" ("Сосач", "Хиккач", если вам угодно). Двач, как и всякая имиджборда разделён на разделы (доски, борды), посвященные различным тематикам -- аниме, видеоигры, литература, религия... Каждая доска состоит из тем (тредов, топиков), которые создаются анонимными (при их желании) пользователями. Каждый тред посвящен обсуждению какого-то конкретного вопроса.

У некоторых разделов есть раздел архив, располагается он по адресу https://2ch.hk/(название раздела)/arch/, например для раздела музыка -- https://2ch.hk/mu/arch/. Если у вас есть минимальные навыки в языке html, а также если вы изучили документацию встроенного класса HTMLParser, то вам будет несложно написать два парсера.

Первый парсер (ArchiveParser) парсит HTML-страницу архива доски, вытягивает из неё ссылки на заархивированные треды, и скармливает их второму парсеру.

Второй парсер (ThreadParser) парсит HTML-страницу заархивированного треда, вытягивает из неё сообщения, складывает их вместе и собирает.

In [1]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
import time
import urllib.request
from html.parser import HTMLParser
from gensim.utils import simple_preprocess

def get_value_by_key(attrs, key):
    for (k, v) in attrs:
        if(k == key):
            return v;
    return None

class ArchiveParser(HTMLParser):
    flag = False
    threads = []
    limit = 200
    def handle_starttag(self, tag, attrs):
        if(self.limit > 0):
            if(tag == 'div'):
                cl = get_value_by_key(attrs, 'class')
                if (cl == 'box-data'):
                    self.flag = True;
            if(self.flag == True and tag == 'a'):
                href = get_value_by_key(attrs, 'href')
                if(len(href)>20):
                    print(href)
                    print(self.limit)
                    thread = parse_thread('https://2ch.hk' + href)
                    if(len(thread) > 10):
                        self.threads.append(thread)
                        self.limit = self.limit - 1
                    thread = []
        

    def handle_endtag(self, tag):
        if(tag == 'div'):
            self.flag = False;

    def handle_data(self, data):
        1+1
        
    def get_threads(self):
        return self.threads
    
    def clean(self):
        self.threads = []
        
parser = ArchiveParser()

def parse_archive(board = '/b/', page_number = 0):
    lines = []
    link = 'https://2ch.hk' + board + 'arch/' + str(page_number) +'.html'
    print(link)
    parser.limit = 100
    url = urllib.request.urlopen(link)
    for line in url.readlines():
        lines.append(line.decode('utf-8'))
    for line in lines:
        parser.feed(line)
    res = parser.get_threads()
    parser.clean()
    return res

In [3]:
class ThreadParser(HTMLParser):
    flag = False
    message = []
    messages = []
            
    def handle_starttag(self, tag, attrs):
        if(tag == 'blockquote'):
            self.flag = True;
            self.message = []
        

    def handle_endtag(self, tag):
        if(tag == 'blockquote'):
            self.flag = False
            if(self.message != []):
                self.messages.append(self.message)
            self.message = []

    def handle_data(self, data):
        if(self.flag):
            self.message.extend(simple_preprocess(data))
            
    def get_messages(self):
        return self.messages
    
    def clear_messages(self):
        flag = False
        self.message = []
        self.messages = []

t_parser = ThreadParser()

def parse_thread (link):
    url = urllib.request.urlopen(link)
    lines = []
    for line in url.readlines():
        lines.append(line.decode('utf-8', errors='ignore'))
    for line in lines:
        t_parser.feed(line)
    res = t_parser.get_messages()
    t_parser.clear_messages()
    #print(res)
    return res

Весьма много кода, верно? Если не потерялись, могли заметить функцию parse_archive, которая парсит страницу архива по доске и номеру страницы.


$\textbf{Задание.}$
Давайте применим её к каким-нибудь доскам. Выберите две доски двача, имеющие архив и скачайте архивы функцией parse_archive.

In [4]:
#TODO: напишите название досок в формате /'доска'/, например /mu/ для Музыки
boards = ['/pr/','/ftb/'] 
threads_by_topic = [parse_archive(board=board) for board in boards]

https://2ch.hk/pr/arch/0.html
/pr/arch/2016-05-04/res/708924.html
100
/pr/arch/2016-05-03/res/708740.html
100
/pr/arch/2016-05-11/res/708703.html
100
/pr/arch/2016-05-03/res/708541.html
99
/pr/arch/2016-05-02/res/708338.html
99
/pr/arch/2016-05-02/res/708024.html
99
/pr/arch/2016-05-02/res/708023.html
99
/pr/arch/2016-05-02/res/708010.html
99
/pr/arch/2016-05-09/res/707960.html
98
/pr/arch/2016-05-12/res/707882.html
97
/pr/arch/2016-05-03/res/707795.html
96
/pr/arch/2016-05-04/res/707611.html
95
/pr/arch/2016-05-11/res/707568.html
94
/pr/arch/2016-05-02/res/707426.html
93
/pr/arch/2016-06-08/res/707395.html
92
/pr/arch/2016-05-01/res/707282.html
91
/pr/arch/2016-05-11/res/707083.html
91
/pr/arch/2016-05-02/res/707079.html
90
/pr/arch/2016-05-01/res/707032.html
89
/pr/arch/2016-04-30/res/707031.html
89
/pr/arch/2016-05-01/res/706875.html
89
/pr/arch/2016-05-07/res/706843.html
88
/pr/arch/2016-05-02/res/706697.html
87
/pr/arch/2016-04-29/res/706407.html
87
/pr/arch/2016-07-26/res/706304.

Разделим наши данны на тренировочые и тестовые. Пусть каждый десятый тред попадает в тест-сет.

In [5]:
data = []
test = []

it = 0
for topic in threads_by_topic:
    for thread in topic:
        full = []
        for post in thread:
            full.extend(post)
        it = it + 1
        if(it % 10 == 0):
            test.append(full)
        else:
            data.append(full)

            

$\textbf{Задание.}$
В русском языке есть множество слов (частицы, междометия, всё что вы хотите), которые никак не отображают смысл слов и являются вспомогательными. Чтобы ваша модель работала лучше -- добавьте стоп-слова в список RUSSIAN_STOP_WORDS или в строку st_str. Эти слова отфильтруются из датасета перед тем, как модель начнет обучаться на датасете.

In [6]:
import nltk
_ = nltk.download("stopwords")
from nltk.corpus import stopwords

russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vladiknaska/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
russian_stopwords[:10]

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [8]:
from gensim.utils import simple_preprocess
from gensim import corpora

RUSSIAN_STOP_WORDS = ['не', 'это', 'что','чем','это','как',
                      'https','нет','op','он','же','так','но',
                      'да','нет','или','и', 'на', "то", "бы", 
                      "все", "ты", "если", "по", "за", "там", 
                      "ну", "уже", "от", "есть","был", "даже", 
                      "было", "www", "com", "youtube", "из",
                      "будет", "mp", "они", "только", "его", "она", 
                      "вот", 'просто', 'watch', 'кто', 'для', 
                      'когда', 'тут', 'мне', 'где', 'мы', 'какой',
                      'может', 'меня', 'до', 'про', 'http', 'раз',
                      'почему', 'тебя', 'ещё', 'их', 'сейчас',
                      'тоже', 'во', 'чтобы', 'этого','без', 'него',
                      'вы','такой', 'можно', 'надо', 'нахуй', 'ли',
                      'потом', 'тред', 'больше', 'лучше', 'хуй', 
                      'сам', 'после', 'со', 'лол', 'быть', 'нужно', 
                      'этом', 'блять', 'бля', 'того', 'ничего', 
                      'потому', 'нибудь', 'этот', 'под', 'через', 
                      'ни', 'себе', 'ему', 'при', 'какие', 'пиздец',
                      'теперь', 'хоть', 'говно', 'тогда', 'блядь', 
                      'кстати', 'че', 'себя', 'конечно', 'типа', 
                      'много', 'том', 'нихуя', 'куда', 'всегда', 'нас', 
                      'тот', 'ведь', 'эти', 'них', 'сука', 'пока', 
                      'более', 'чего', 'html', 'были', 'всех',
                      'была', 'например', 'тем', 'ru', 'зачем', 
                      'либо', 'вроде', 'всего', 'вопрос', 'php', 
                      'против', 'здесь', 'ее', 'значит', 'совсем', 
                      'сколько', 'им', 'org', 'именно', 'эту']

st_str = ('которых которые твой которой которого сих ком свой твоя этими'
          ' слишком нами всему будь саму чаще ваше сами наш затем еще'
          ' самих наши ту каждое мочь весь этим наша своих оба который'
          ' зато те этих вся ваш такая теми ею которая нередко каждая '
          'также чему собой самими нем вами ими откуда такие тому та '
          'очень сама нему алло оно этому кому тобой таки твоё каждые '
          'твои мой нею самим ваши ваша кем мои однако сразу свое ними '
          'всё неё тех хотя всем тобою тебе одной другие этао само эта '
          'буду самой моё своей такое всею будут своего кого свои мог '
          'нам особенно её самому наше кроме вообще вон мною никто это')

RUSSIAN_STOP_WORDS.extend(st_str.split(' '))
RUSSIAN_STOP_WORDS.extend(russian_stopwords)

data = [list(filter(lambda word: not word in RUSSIAN_STOP_WORDS, piece)) for piece in data]
id2word = corpora.Dictionary(data)

Создадим словарь и на его основе преобразуем слова в их id.

In [9]:
id2word = corpora.Dictionary(data)

# Create Corpus
texts = data

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


Обучим LDA-модель, используя библиотеку gensim. Зададим число тем равно числу скачанных досок.

In [10]:
from gensim.models import LdaModel

model = LdaModel(corpus, id2word=id2word, num_topics=len(threads_by_topic))

Теперь получим топ-10 самых используемых в каждой теме слов.

$\textbf{Задание.}$
Оцените насколько хорошо модель разделила темы.

In [11]:
for i in range(len(threads_by_topic)):
    print([id2word[id[0]] for id in model.get_topic_terms(topicid = i, topn = 10)])

['код', 'время', 'делать', 'сделать', 'динамо', 'года', 'россии', 'евро', 'матч', 'лет']
['лет', 'сделать', 'код', 'россии', 'время', 'делать', 'евро', 'россия', 'сборной', 'работает']


Теперь прогоним тестовые треды на модели. Тестовый датасет разделен на n равных частей по 20 тредов, i-ая соответствует i-й доске.

In [12]:
other_corpus = [id2word.doc2bow(text) for text in [list(filter(lambda word: not word in RUSSIAN_STOP_WORDS, piece)) for piece in test]]

vector = [model[unseen_doc] for unseen_doc in other_corpus]
print(vector[0]) #вероятности принадлежности 0-го тестового треда в ту или иную тему

[(0, 0.2870672), (1, 0.7129328)]


In [13]:
i = 0

for res in vector:
    max_it = 0
    if(len(res) > 0):
        for it in range(1, len(res)):
            if(res[max_it][1] < res[it][1]):
                max_it = it
        print("Text #" + str(i) + ", topic #" + str(max_it) + str(", prob = " + str(res[max_it][1])))
    i = i + 1

Text #0, topic #1, prob = 0.7129328
Text #1, topic #0, prob = 0.7574346
Text #2, topic #1, prob = 0.6016699
Text #3, topic #1, prob = 0.5844992
Text #4, topic #1, prob = 0.62229055
Text #5, topic #1, prob = 0.5133282
Text #6, topic #0, prob = 0.6036269
Text #7, topic #1, prob = 0.88308245
Text #8, topic #1, prob = 0.6279996
Text #9, topic #1, prob = 0.5479618
Text #10, topic #0, prob = 0.5413816
Text #11, topic #0, prob = 0.5538754
Text #12, topic #0, prob = 0.7733906
Text #13, topic #1, prob = 0.59811765
Text #14, topic #0, prob = 0.73121154
Text #15, topic #0, prob = 0.6736597
Text #16, topic #0, prob = 0.69949967
Text #17, topic #0, prob = 0.6734555
Text #18, topic #0, prob = 0.7103451
Text #19, topic #0, prob = 0.7124265


$\textbf{Задание.}$

Оцените результаты работы модели на тест сете. Если модель разделили данные плохо -- объясните, почему?

## Часть 2. А теперь нормальный датасет.

А теперь давайте воспользуемся более стандартным датасетом библиотеки sklreatn -- 20newsgroups, посвященную статьям на различные темы. Выберем 6 -- Атеизм, яблочное железо, автомобили, хоккей, космос, христианство, ближний восток.

In [14]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism',
 'comp.sys.mac.hardware',
 'rec.autos',
 'rec.sport.hockey',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.mideast']

newsgroups_train = fetch_20newsgroups(subset='train', 
                                      remove=('headers', 'footers', 'quotes'), 
                                      categories=categories)

In [15]:
list(newsgroups_train.target_names)

['alt.atheism',
 'comp.sys.mac.hardware',
 'rec.autos',
 'rec.sport.hockey',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.mideast']

$\textbf{Задание}$

Найдите библиотечный или опишите свой список ENGSLISH_STOP_WORDS, убирающий не несущие никакого смысла английские слова.

In [16]:
ENGLISH_STOP_WORDS = stopwords.words('english') #TODO

print('the' in ENGLISH_STOP_WORDS) 

data = [list(filter(lambda word: not word in ENGLISH_STOP_WORDS, simple_preprocess(piece)))
                    for piece in newsgroups_train.data]

True


$\textbf{Большое задание 1.}$

Для списка data создайте словарь id2word. Получите преобразованный TermDocumentFrequency список corpust и обучите на нем LDA модель.

In [17]:
from gensim.models import LdaModel, LsiModel
from gensim import corpora

print(data[0])
id2word = corpora.Dictionary(data)
# Create Corpus
texts = data
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

model = LdaModel(corpus, id2word=id2word, 
                 num_topics=len(categories), iterations=400) #TODO

['considering', 'adding', 'floptical', 'drive', 'current', 'system', 'would', 'like', 'know', 'floptical', 'drives', 'recommended', 'quality', 'performance', 'preference', 'would', 'floptical', 'drives', 'capable', 'handling', 'floppies', 'handling', 'floppies', 'necessity', 'far', 'know', 'bit', 'iomega', 'floptical', 'infinity', 'floptical', 'drives', 'comments', 'recommendations', 'either', 'floptical', 'drives', 'worth', 'looking', 'purchased', 'mail', 'order', 'places', 'etc', 'thanks', 'advance', 'please', 'send', 'replies', 'directly', 'umsoroko', 'ccu', 'umanitoba', 'ca']
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 4), (12, 1), (13, 1), (14, 1), (15, 2), (16, 6), (17, 2), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2)]]


In [18]:
#Выведем получившийся список тем:
for i, cat in enumerate(categories):
    print(f'{cat.title()}: ')
    print([id2word[idx[0]] for idx in model.get_topic_terms(topicid=i, topn=25)])

Alt.Atheism: 
['finnish', 'chart', 'abstinence', 'checking', 'laws', 'one', 'reasonable', 'failure', 'however', 'wake', 'catholic', 'representation', 'invaders', 'warrant', 'evils', 'seems', 'bottom', 'would', 'predicted', 'world', 'armies', 'country', 'government', 'religious', 'everywhere']
Comp.Sys.Mac.Hardware: 
['think', 'gt', 'manta', 'rate', 'use', 'sure', 'pretty', 'early', 'sold', 'name', 'kadett', 'would', 'like', 'actual', 'one', 'top', 'sex', 'students', 'even', 'mid', 'list', 'bottom', 'high', 'impressed', 'control']
Rec.Autos: 
['turkey', 'govern', 'volunteers', 'rate', 'kurds', 'failure', 'bristol', 'caucasus', 'armenian', 'ability', 'pot', 'russian', 'arfa', 'top', 'irregular', 'prevention', 'ambassador', 'power', 'safest', 'one', 'london', 'disaster', 'source', 'eastern', 'races']
Rec.Sport.Hockey: 
['ottawa', 'would', 'surprise', 'smileys', 'absolute', 'psuvm', 'hirsch', 'shades', 'rangers', 'team', 'cares', 'american', 'go', 'could', 'one', 'edu', 'space', 'patrick',

$\textbf{Большое задание 2.}$

В соответствии с тренировочными, обработайте тестовые данные.

Напишите функцию, которая с помощью модели возвращает наиболее вероятный id темы. С помощью F-меры оцените правильность работы модели.

In [19]:
newsgroups_test = fetch_20newsgroups(subset='test', 
                                     remove=('headers', 'footers', 'quotes'), 
                                     categories = categories)
#TODO: YOUD CODE

In [20]:
def create_test_corpus(dataset, stopwords):
    data = [list(filter(lambda word: not word in stopwords, simple_preprocess(piece))) \
            for piece in dataset.data]
    id2word = corpora.Dictionary(data)
    corpus = [id2word.doc2bow(text) for text in data]
    return corpus

In [21]:
test_corpus = create_test_corpus(newsgroups_test, ENGLISH_STOP_WORDS)

In [22]:
def predict(model, text):
    vector = model[text]
    return max(vector, key=lambda x:x[1])

In [23]:
def find_f_score(per, y_pred, y_true):
    d = {0:per[0], 1:per[1], 2:per[2], 3:per[3], 
         4:per[4], 5:per[5], 6:per[6]}
    new_vec = [d.get(n, n) for n in y_pred]
    f1 = f1_score(y_true, new_vec, average='weighted')
#     acc = accuracy_score(y_true, new_vec)
    return f1

In [24]:
import itertools
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm

permuts = list(itertools.permutations([0, 1, 2, 3, 4, 5, 6]))
y_pred = [predict(model, sample)[0] for sample in test_corpus]
y_true = newsgroups_test['target'].tolist()

res = {}
for per in tqdm(permuts):
    res[per] = find_f_score(per, y_pred, y_true)

100%|██████████| 5040/5040 [00:37<00:00, 133.91it/s]


In [26]:
import operator
res_perm = max(res.items(), key=operator.itemgetter(1))[0]
res[res_perm]

0.1696403211641113

In [27]:
for i, true_id in zip(range(len(categories)), res_perm):
    print(f'{categories[true_id].title()}: ')
    print([id2word[idx[0]] for idx in model.get_topic_terms(topicid=i, topn=25)])

Sci.Space: 
['finnish', 'chart', 'abstinence', 'checking', 'laws', 'one', 'reasonable', 'failure', 'however', 'wake', 'catholic', 'representation', 'invaders', 'warrant', 'evils', 'seems', 'bottom', 'would', 'predicted', 'world', 'armies', 'country', 'government', 'religious', 'everywhere']
Rec.Autos: 
['think', 'gt', 'manta', 'rate', 'use', 'sure', 'pretty', 'early', 'sold', 'name', 'kadett', 'would', 'like', 'actual', 'one', 'top', 'sex', 'students', 'even', 'mid', 'list', 'bottom', 'high', 'impressed', 'control']
Talk.Politics.Mideast: 
['turkey', 'govern', 'volunteers', 'rate', 'kurds', 'failure', 'bristol', 'caucasus', 'armenian', 'ability', 'pot', 'russian', 'arfa', 'top', 'irregular', 'prevention', 'ambassador', 'power', 'safest', 'one', 'london', 'disaster', 'source', 'eastern', 'races']
Soc.Religion.Christian: 
['ottawa', 'would', 'surprise', 'smileys', 'absolute', 'psuvm', 'hirsch', 'shades', 'rangers', 'team', 'cares', 'american', 'go', 'could', 'one', 'edu', 'space', 'patri

## Experiments with number of categories

In [32]:
categories = ['alt.atheism',
 'comp.sys.mac.hardware']

newsgroups_train = fetch_20newsgroups(subset='train', 
                                      remove=('headers', 'footers', 'quotes'), 
                                      categories=categories)

In [33]:
data = [list(filter(lambda word: not word in ENGLISH_STOP_WORDS, simple_preprocess(piece)))
                    for piece in newsgroups_train.data]

In [34]:
id2word = corpora.Dictionary(data)
texts = data
corpus = [id2word.doc2bow(text) for text in texts]

model = LdaModel(corpus, id2word=id2word, 
                 num_topics=len(categories), iterations=700)

In [35]:
for i, cat in enumerate(categories):
    print(f'{cat.title()}: ')
    print([id2word[idx[0]] for idx in model.get_topic_terms(topicid=i, topn=25)])

Alt.Atheism: 
['one', 'god', 'would', 'people', 'know', 'apple', 'think', 'like', 'get', 'anyone', 'also', 'problem', 'new', 'could', 'well', 'good', 'see', 'say', 'mac', 'time', 'jesus', 'even', 'argument', 'true', 'something']
Comp.Sys.Mac.Hardware: 
['one', 'would', 'mac', 'people', 'like', 'system', 'god', 'drive', 'know', 'think', 'use', 'could', 'bit', 'many', 'well', 'mb', 'even', 'time', 'also', 'get', 'way', 'say', 'atheism', 'scsi', 'believe']


In [38]:
newsgroups_test = fetch_20newsgroups(subset='test', 
                                     remove=('headers', 'footers', 'quotes'), 
                                     categories = categories)

test_corpus = create_test_corpus(newsgroups_test, ENGLISH_STOP_WORDS)

In [39]:
y_pred = [predict(model, sample)[0] for sample in test_corpus]
y_true = newsgroups_test['target'].tolist()

d = {0:1, 1:0}
new_vec = [d.get(n, n) for n in y_pred]
f1_score(y_true, new_vec)

0.46579330422125187

In [40]:
f1_score(y_true, y_pred)

0.5717916137229987

## Experiments with preprocessing

In [63]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

def lemmatize_stemming(text):
    stemmer = PorterStemmer()
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vladiknaska/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [64]:
categories = ['alt.atheism',
 'comp.sys.mac.hardware']

newsgroups_train = fetch_20newsgroups(subset='train', 
                                      remove=('headers', 'footers', 'quotes'), 
                                      categories=categories)

In [65]:
data = [preprocess(text) for text in newsgroups_train.data]

In [67]:
id2word = corpora.Dictionary(data)
texts = data
corpus = [id2word.doc2bow(text) for text in texts]
model = LdaModel(corpus, id2word=id2word, 
                 num_topics=len(categories), iterations=700)

In [68]:
newsgroups_test = fetch_20newsgroups(subset='test', 
                                     remove=('headers', 'footers', 'quotes'), 
                                     categories = categories)

test_corpus = create_test_corpus(newsgroups_test, ENGLISH_STOP_WORDS)

In [71]:
len(y_true)

704

In [72]:
data = [preprocess(text) for text in newsgroups_test.data]
id2word = corpora.Dictionary(data)
corpus = [id2word.doc2bow(text) for text in data]

In [84]:
y_pred = [predict(model, sample)[0] for sample in corpus]
y_true = newsgroups_test['target'].tolist()

d = {0:1, 1:0}
new_vec = [d.get(n, n) for n in y_pred]
f1_score(y_true, new_vec)

0.5881006864988557